In [ ]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm

In [ ]:
# 데이터 임포팅 
df = pd.read_excel('offerTrain.xlsx')
df.dropna(inplace=True)

In [ ]:
df_copy = df.drop_duplicates(['상품명'])

### 간단한 표준화

In [ ]:
## 토큰들 문자열로 결합
def tokenToString(token_list):
    res = []
    for token in token_list:
      s = ' '.join(token)
      res.append(s)
    return res

## []안에 문자가 있다? (브랜드명), ()안에 한글이 있다? or 숫자뒤에 한글/영어가 온다? (단위)
## & , + -> 거의 세트메뉴이다 -> 살리기
## 1. 특문 별 나오는 빈도 체크
## 2. 뭘 놔둘지 확인

pat = re.compile("\[[^)]*\]|\([가-힣]*\)|[0-9]+[a-zA-Z가-힣]*") 
# pat = re.compile("\([가-힣]*\)|[0-9]+[a-zA-Z가-힣]*") 

## 특수문자 제외
sp = re.compile("[^가-힣|&|+]")

## 지우는 전처리
def remove_eda(df, col):
  token_list = []
  for t in tqdm(df[col]):
    if t:
      t1 = pat.sub("", str(t))
      text = sp.sub("", t1).lower()
      res = text.split()
      token_list.append(res)
    else:
      token_list.append(t)
  result = tokenToString(token_list)
  df[col] = result
  return df

In [ ]:
df_copy = df_copy[['상점명','상품명','표준상품명']]

In [ ]:
for col in df_copy.columns:
  df_copy = remove_eda(df_copy, col)

100%|████████████████████████████████████████████████████████████████████████| 75708/75708 [00:00<00:00, 586164.61it/s]


In [ ]:
df_copy

,상점명,상품명,표준상품명
0,단팥빵불광점,,
1,단팥빵불광점,,
2,미각상점강남역지하상가점,,
10,행복한하루,,
11,에그드랍익산모현점,,
...,...,...,...
428671,,뉴질랜드해산물리조또,뉴질랜드해산물리조또
428677,,사료양고기리조또,사료양고기리조또
428678,,사슴힘줄,사슴힘줄
428679,,쇠고기져키,쇠고기져키


2023-01-17

- [해결] 빈도수 비율 없이, 그냥 맨 뒷 토큰을 사용해보는 실험 해보기

- [해결] soynlp를 통한 토큰화 시도


2023-01-24

- [해결] 커스텀 사전 제작 

- [해결] 맨 뒷 단어 중심 알고리즘 예외처리

- [해결] 초성 기준의 표준화 작업

## loanword 기법 적용한 표준화

In [ ]:
kor_begin = 44032
kor_end = 55203
chosung_base = 588
jungsung_base = 28
jaum_begin = 12593
jaum_end = 12622
moum_begin = 12623
moum_end = 12643

chosung_list = [ 'ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 
        'ㅅ', 'ㅆ', 'ㅇ' , 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

jungsung_list = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 
        'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 
        'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 
        'ㅡ', 'ㅢ', 'ㅣ']

jongsung_list = [
    ' ', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ',
        'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 
        'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 
        'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

jaum_list = ['ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄸ', 'ㄹ', 
              'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 
              'ㅃ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

moum_list = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 
              'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']

def compose(chosung, jungsung, jongsung):
    char = chr(
        kor_begin +
        chosung_base * chosung_list.index(chosung) +
        jungsung_base * jungsung_list.index(jungsung) +
        jongsung_list.index(jongsung)
    )
    return char

def decompose(c):
    if not character_is_korean(c):
        return c
    i = ord(c)
    if (jaum_begin <= i <= jaum_end):
        return (c, ' ', ' ')
    if (moum_begin <= i <= moum_end):
        return (' ', c, ' ')

    # decomposition rule
    i -= kor_begin
    cho  = i // chosung_base
    jung = ( i - cho * chosung_base ) // jungsung_base 
    jong = ( i - cho * chosung_base - jung * jungsung_base )

    if jongsung_list[jong] == ' ': ## 종성이 없을 경우, 문자를 넣지 않는다
      return (chosung_list[cho], jungsung_list[jung])

    return (chosung_list[cho], jungsung_list[jung], jongsung_list[jong])

def character_is_korean(c):
    i = ord(c)
    return ((kor_begin <= i <= kor_end) or
            (jaum_begin <= i <= jaum_end) or
            (moum_begin <= i <= moum_end))
    


# 제4항 파열음 표기에는 된소리를 쓰지 않는 것을 원칙으로 한다.
## 된소리의 경우 id값에서 -1을 해주면 예사소리가 된다. 

## 거센소리로 통일시키는게 낫다는 판단이 들었다.

## 카라멜마끼아또 -> 카라멜마기아도 (X)

loanword_cho_list = ['ㄲ', 'ㄸ', 'ㅃ', 'ㅆ', 'ㅉ']

def loanword_cho(cho):
  if cho in loanword_cho_list:
    if cho == loanword_cho_list[0]:
      cho = 'ㅋ'
    elif cho == loanword_cho_list[1]:
      cho = 'ㅌ'
    elif cho == loanword_cho_list[2]:
      cho = 'ㅍ'
    elif cho == loanword_cho_list[3]:
      cho = 'ㅅ' ## ㅅ의 경우, 거센소리가 없으므로 예사소리로 통일시킨다.
    elif cho == loanword_cho_list[4]:
      cho = 'ㅊ'
  
  return cho

## 모음의 일반화, 의미가 통한다고 가정하고 2개가 합쳐진 모음은 하나로 뭉침
## 이중모음 11개 + 단모음 1개를 단모음으로 통일
def loanword_jung(moum):
  if moum in ['ㅐ','ㅖ','ㅒ']:
    moum = 'ㅔ'
  if moum in ['ㅞ','ㅙ']:
    moum = 'ㅚ'
  elif moum in ['ㅘ', 'ㅑ']:
    moum = 'ㅏ'
  elif moum in ['ㅝ', 'ㅕ']:
    moum = 'ㅓ'
  elif moum in ['ㅟ', 'ㅢ']:
    moum = 'ㅣ'
  elif moum in ['ㅠ']:
    moum = 'ㅜ'
  elif moum in ['ㅛ']:
    moum = 'ㅗ'
  
  return moum


## 제3항 받침에는 ‘ㄱ, ㄴ, ㄹ, ㅁ, ㅂ, ㅅ, ㅇ’만을 쓴다.
def loanword_jong(jamo):
  if jamo in ['ㄲ', 'ㄳ', 'ㄺ', 'ㅋ']:
    jamo = 'ㄱ'
  elif jamo in ['ㄵ', 'ㄶ']:
    jamo = 'ㄴ'
  elif jamo in ['ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ']:
    jamo = 'ㄹ'
  elif jamo in ['ㅄ', 'ㄶ','ㅍ']:
    jamo = 'ㅂ'
  elif jamo in ['ㄷ','ㅆ','ㅈ','ㅊ','ㅌ','ㅎ']:
    jamo = 'ㅅ'
  
  return jamo

In [ ]:
import hgtk
from tqdm import tqdm

def jamo_to_word(jamo_sequence):
    tokenized_jamo = []
    index = 0

# 1. 입력값 : '남동생'

    while index < len(jamo_sequence):
    # 문자가 정상적인 한글이 아닐 경우
        if not hgtk.checker.is_hangul(jamo_sequence[index]):
            tokenized_jamo.append(jamo_sequence[index])
            index = index + 1

    # 문자가 정상적인 한글이라면 초/중/종성으로 나눔
        else:
            tokenized_jamo.append(jamo_sequence[index:index + 3])
            index = index + 3

# 2. 자모단위 토큰화 완료
# tokenized_jamo : ['남', '동', '생]

        word = ''
        try:
            for jamo in tokenized_jamo:
            # 초, 중, 종성의 묶음으로 추정되는 경우
                if len(jamo) == 3:
                    if jamo[2] == "-":
                    # 종 성 이 존 재 하 지 않 는 경 우
                        word = word + hgtk.letter.compose(jamo[0], jamo[1])
                    else:
                # 종 성 이 존 재 하 는 경 우
                        word = word + hgtk.letter.compose(jamo[0], jamo[1], jamo[2])
                # 한 글 이 아 닌 경 우
                else:
                    word = word + jamo

            # 복 원 중 (hgtk.letter.compose) 에 러 발 생 시 초 기 입 력 리 턴 .
            # 복 원 이 불 가 능 한 경 우 예 시 ) 'ㄴ!ㄷㅗㅇㅅㅐㅇ'
        except Exception as exception:
            if type(exception).__name__ == 'NotHangulException':
                return jamo_sequence

        # 3. 단 어 로 복 원 완 료
        # word : ' 남 동 생 '

    return word

## 단어를 loanword 기법 적용 후 변환하는 함수
def word_loanword_transform(token):
    def to_special_token(jamo):
      if not jamo:
        return '-'
      else:
        return jamo
    decomposed_token = ''

    for char in token:
        try:
        # char( 음 절 ) 을 초 성 , 중 성 , 종 성 으 로 분 리
            cho, jung, jong = hgtk.letter.decompose(char)

            # 자 모 가 빈 문 자 일 경 우 특 수 문 자 - 로 대 체
            cho = to_special_token(cho)
            cho = loanword_cho(cho)
            jung = to_special_token(jung)
            jung = loanword_jung(jung)
            jong = to_special_token(jong)
            jong = loanword_jong(jong)
            decomposed_token = decomposed_token + cho + jung + jong

        # 만 약 char( 음 절 ) 이 한 글 이 아 닐 경 우 자 모 를 나 누 지 않 고 추 가
        except Exception as exception:
            if type(exception).__name__ == 'NotHangulException':
                decomposed_token += char

    # 단 어 토 큰 의 자 모 단 위 분 리 결 과 를 추 가
    return jamo_to_word(decomposed_token)


In [ ]:
menu_list = df_copy['상품명'].tolist()

for i, menu in enumerate(menu_list):
  if str(type(menu)) != "<class 'str'>":
    print(menu)
    print(type(menu))
    menu_list[i] = str(menu)


## loanword 적용 시, 주석 제거

# for i, menu in enumerate(menu_list):
#   menu_list[i] = word_loanword_transform(menu)

In [ ]:
corpus = pd.read_csv('nlp_custom_dict_v4.txt', header=None)

In [ ]:
## loanword 적용 시 주석 제거
#corpus[0] = corpus[0].apply(word_loanword_transform)

corpus = list(corpus[0])

In [ ]:
from soynlp.tokenizer import MaxScoreTokenizer, RegexTokenizer

tokenizer = RegexTokenizer() ## Regex 기준 토크나이저, 띄어쓰기가 있거나 이어지는 단어가 한국어가 아닐 경우 토큰화

corpus_list = []

for word in corpus:
  corpus_list.append(tokenizer.tokenize(word))

C:\Users\user\anaconda3\lib\site-packages\soynlp\tokenizer\_tokenizer.py:19: FutureWarning: Possible nested set at position 13
  ('english & latin', re.compile(u"[a-zA-ZÀ-ÿ]+[[`']?s]*|[a-zA-ZÀ-ÿ]+", re.UNICODE))


In [ ]:
corpus_dict = {}

for tokens in corpus_list:
  tokens.sort(key=len)
  for i, token in enumerate(tokens):
    corpus_dict[token] = round(1/len(tokens[i]), 1)
    

    ## MaxScoreTokenizer에 사용할 커스텀 사전, 점수를 넣을 경우 해당 단어를 토큰으로 분리하는 가중치를 지정
    ## 길이가 짧은 단어를 기준으로 1점 부터 시작해 길이 비례로 작은 점수를 지정해준다
    ## 
    ## 서울대학교 라는 단어가 있는데, 서울 :1 대학교 : 1 서울대학교 : 1 이처럼 점수가 같을 경우
    ## 단어가 긴 서울대학교를 우선으로 지정해주는 방식

In [ ]:
tokenizer = MaxScoreTokenizer(scores=corpus_dict) ## 위에서 만든 사전을 점수 계산으로 지정

In [ ]:
from tqdm import tqdm

menu_list = [tokenizer.tokenize(word) for word in tqdm(menu_list)]

100%|█████████████████████████████████████████████████████████████████████████| 75708/75708 [00:01<00:00, 56481.84it/s]


In [ ]:
## 커스텀 사전의 토큰들만 리스트로 추출
corpus_dict_list = list(corpus_dict.keys())

In [ ]:
from collections import Counter

df_copy['tokens'] = menu_list
df_copy.reset_index(drop=True, inplace=True)
# Counter 객체는 리스트요소의 값과 요소의 갯수를 카운트 하여 저장하고 있습니다.
# 카운터 객체는 .update 메소드로 계속 업데이트 가능합니다.
word_counts = Counter()

# 토큰화된 각 리뷰 리스트를 카운터 객체에 업데이트 합니다. 
for tokens in df_copy['tokens']:
  for token in tokens:
    if token in corpus_dict_list: ## 사전에 있는 경우만 빈도수를 체크
      word_counts.update([token])

# 가장 많이 존재하는 단어 순으로 10개를 나열합니다
word_counts.most_common(10)

[('라떼', 4845),
 ('치즈', 3871),
 ('크림', 2916),
 ('세트', 2841),
 ('초코', 2223),
 ('추가', 2037),
 ('티', 1979),
 ('딸기', 1654),
 ('에이드', 1463),
 ('치킨', 1429)]

In [ ]:
## 가장 적게 존재하는 단어 순으로 정렬합니다. 10개만 나타내는 경우
word_counts.most_common()[:-10:-1] 

[('양귀', 1),
 ('그랜드웨딩티', 1),
 ('이엠티', 1),
 ('힐스트림', 1),
 ('담수전용', 1),
 ('제오팩', 1),
 ('마사무네', 1),
 ('중성펜', 1),
 ('추카', 1)]

In [ ]:
len(word_counts)

3540

In [ ]:
word_counts_sorted = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)

In [ ]:
token_list = word_counts.keys()

In [ ]:
len_1_tokens = []

for token in token_list:
  if len(token) <= 1:
    len_1_tokens.append(token)

In [ ]:
len(token_list)

3540

In [ ]:
len(len_1_tokens)

136

# 사전에 들어있는 단어(음식 이름)만을 이용해서 카테고리로 분류

In [ ]:
def levenshtein(s1, s2, cost=None, debug=False):
    if len(s1) < len(s2):
        return levenshtein(s2, s1, debug=debug)

    if len(s2) == 0:
        return len(s1)

    if cost is None:
        cost = {}

    # changed
    def substitution_cost(c1, c2):
        if c1 == c2:
            return 0
        return cost.get((c1, c2), 1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            # Changed
            substitutions = previous_row[j] + substitution_cost(c1, c2)
            current_row.append(min(insertions, deletions, substitutions))

        if debug:
            print(current_row[1:])

        previous_row = current_row

    return previous_row[-1]

def custom_jamo_levenshtein(s1, s2, debug=False):
    s1_ = ''.join([comp for c in s1 for comp in decompose(c)])
    s2_ = ''.join([comp for c in s2 for comp in decompose(c)])

    return levenshtein(s1_, s2_)/3 

In [ ]:
## 빈도수 기준 내림차순 정렬한 토큰 사전 리스트
word_counts_list = [x[0] for x in word_counts_sorted]

In [ ]:
## 표준화 결과를 저장할 딕셔너리
std_dict = {}

In [ ]:
import pickle

## 미리 저장된 std_dict이 있다면 로드해서 사용
with open('std_dict.pkl','rb') as f:
  std_dict = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'std_dict.pkl'

In [ ]:
def jamo_std(word, count_threshold=1, score_threshold=0.5):
  ## count_threshold : 점수 계산 기준 빈도수 기본값 : 1 / 빈도수가 1보다 큰 단어만 점수 계산
  ## score_threshold : 점수 계산 시 범위 기준값 기본값 : 0.5 / 최소점수가 0일 경우 0 + 0.5 의 범위까지 탐색

  # 표준화 결과가 있는 단어일 경우 바로 변환
  if std_dict.get(word) is not None:
    return std_dict[word]

  scores = [(custom_jamo_levenshtein(word, x[0]),x[0], x[1]) for x in word_counts_sorted if x[1] > count_threshold]

  scores_sorted = sorted(scores, key=lambda x: x[0]) ## levenshtein 비용 기준으로 오름차순으로 정렬
  min_score = scores_sorted[0][0] ## 가장 작은 점수(단어 중 비용이 가장 작은 값)

  scores = [x for x in scores_sorted if x[0] <= min_score + score_threshold] ## 최소 비용 + 범위 임계값

  scores_sorted = sorted(scores, key=lambda x: x[2], reverse=True) ## 점수가 같을 경우 빈도수 높은 것을 선택하기 위해 정렬


  std_word = scores_sorted[0][1]
  std_dict[word] = std_word
  
  return std_word

빈도수 기반이라 아이스 / 참이슬 같은 경우에서 참이슬이라는 고유명사가 제대로 반영되지 않는 부분이 있다.

임계점을 줄이면 해결되지만 그럴 경우 함바그 - 함박 같은 사례를 해결하지 못한다.

- [미해결] 차이슬 -> 참이슬의 문제

In [ ]:
def rules(tokens, text):
  if text in tokens:
    tok=[]
    idx = tokens.index(text)

    if len(text) == 1: # 건, 생 처리
      if idx == len(tokens) - 1:
          del tokens[idx]
          return tokens
      next_word = tokens[idx+1]
      if next_word not in corpus: # 다음 단어가 단어사전에 없다면 
        del tokens[idx:idx+2]     # -> 쓰레기 데이터로 판단 -> 둘다 날려주기
      else:
        tokens[idx] = '('+text+')'
        tok.append(tokens[idx] + tokens[idx+1])
        tok.extend(tokens[idx+2:])
        tokens = tok
        corpus_dict_list.append(tok[0])
    else: # 시럽, 소스 처리
      if idx == 0:
        return tokens
      tok.append(tokens[idx-1] + tokens[idx])
      tok.extend(tokens[idx+1:])
      tokens = tok
      corpus_dict_list.append(tok[0])
    return tokens
  else:
    return tokens

In [ ]:
from tqdm import tqdm
import ray
import pickle

menu_cluster = {} ## 메뉴 분류 dict
OOV_list = Counter() ## OOV 빈도수 사전
std_oov_list = set([])
option_dict = ['사이즈', '변경', '추가', '세트'] ## "옵션"에 해당하는 사전

num_cpus = 12
ray.init(num_cpus=num_cpus, ignore_reinit_error=True)

2023-01-28 20:24:40,746	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.9.13
Ray version:,2.2.0
Dashboard:,http://127.0.0.1:8265


In [ ]:
@ray.remote
def sandevistan_std(df_copy, corpus_dict_list, option_dict, OOV_list, menu_cluster):

    for i, tokens in tqdm(enumerate(df_copy['tokens'])):
      menu_c = ''
      menu_origin = df_copy['상품명'][i]

      ## 시럽, 소스는 요소와 붙이기
      tokens = rules(tokens, '시럽')
      tokens = rules(tokens, '소스')
      ## (건), (생) 처리
      ## 건, 생 단어사전에 추가 -> 잡아내기
      ## 건/생 뒤 단어가 사전에 없는 경우 -> 건/생+뒤 단어 같이 날려버리기
      ## 있는 경우 괄호처리
      tokens = rules(tokens, '건')
      tokens = rules(tokens, '생')

      ##라떼는 요소와 띄어쓰기, 카페라떼 제외
      # -> 단어사전 수정

      for token in tokens:
        if token in corpus_dict_list: ## 사전에 있는 경우
          if token in option_dict: ## 옵션에 해당하는 토큰이 있을 경우, "-" 으로 표준화
            menu_c = "-"
            break

          token = jamo_std(token) ## 자모 단위 표준화
          menu_c += token + " " ## 토큰 마다 띄어쓰기
        else:
          #token_std = jamo_std(token)
          OOV_list.update([token]) ## 토큰이 사전에 없을 경우 OOV 리스트에 삽입, (원본, 표준화 이후) 형태로 삽입
          #std_oov_list.add((token, token_std))

      ## 내부의 어떤 토큰도 사전에 없을 경우 "-" 으로 표준화
      if menu_c == '':
        menu_c = "-"

      menu_cluster[menu_origin] = menu_c.rstrip() ## 토큰이 사전에 있는 단어(음식 이름)일 경우만 넣는다.
      ## 맨 뒤의 공백은 제거하기 위해 rstrip()을 사용
    return 


In [ ]:
def ray_std():
    
    corpus_id = ray.put(df_copy)
    sandevistan_ids = []
    for i in range(num_cpus):
        sandevistan_ids.append(sandevistan_std.remote(corpus_id))

    ray.get(sandevistan_ids)

In [ ]:
ray_std()

0it [00:00, ?it/s]tan_std pid=12564) 
0it [00:00, ?it/s]tan_std pid=14252) 
0it [00:00, ?it/s]tan_std pid=13960) 
0it [00:00, ?it/s]tan_std pid=11412) 
0it [00:00, ?it/s]tan_std pid=7192) 
0it [00:00, ?it/s]tan_std pid=12220) 
0it [00:00, ?it/s]tan_std pid=4112) 
0it [00:00, ?it/s]tan_std pid=13100) 
0it [00:00, ?it/s]tan_std pid=12500) 
0it [00:00, ?it/s]tan_std pid=104) 
0it [00:00, ?it/s]tan_std pid=7040) 
0it [00:00, ?it/s]tan_std pid=7152) 
127it [00:00, 340.21it/s] pid=11412) 
127it [00:00, 293.06it/s] pid=12564) 
127it [00:00, 361.68it/s] pid=12500) 
127it [00:00, 351.55it/s] pid=7192) 
127it [00:00, 338.56it/s] pid=14252) 
127it [00:00, 361.63it/s] pid=12220) 
127it [00:00, 340.19it/s] pid=4112) 
127it [00:00, 345.93it/s] pid=13960) 
127it [00:00, 361.75it/s] pid=104) 
127it [00:00, 301.32it/s] pid=13100) 
127it [00:00, 374.61it/s] pid=7152) 
127it [00:00, 318.13it/s] pid=7040) 
157it [00:03, 38.59it/s]  pid=12564) 
161it [00:03, 39.57it/s]  pid=14252) 
162it [00:03, 39.73it/s]

234it [00:09, 27.90it/s]d pid=7192) 
220it [00:09, 12.93it/s]d pid=7152) 
234it [00:10, 29.18it/s]d pid=13960) 
222it [00:10, 13.31it/s]d pid=7152) 
238it [00:10, 16.25it/s]d pid=12564) 
234it [00:10, 24.26it/s]d pid=7152) 
238it [00:10, 15.30it/s]d pid=4112) 
239it [00:10, 15.57it/s]d pid=11412) 
238it [00:10, 15.95it/s]d pid=12500) 
238it [00:10, 15.97it/s]d pid=13100) 
238it [00:10, 15.55it/s]d pid=7192) 
238it [00:10, 14.36it/s]d pid=14252) 
239it [00:10, 14.66it/s]d pid=104) 
238it [00:10, 16.23it/s]d pid=7040) 
238it [00:10, 14.21it/s]d pid=12220) 
241it [00:10, 13.14it/s]d pid=12564) 
241it [00:10, 12.94it/s]d pid=4112) 
238it [00:10, 14.86it/s]d pid=13960) 
241it [00:10, 13.45it/s]d pid=12500) 
241it [00:10, 13.25it/s]d pid=13100) 
241it [00:10, 12.94it/s]d pid=14252) 
246it [00:11, 14.32it/s]d pid=11412) 
241it [00:10, 13.09it/s]d pid=7192) 
241it [00:10, 13.72it/s]d pid=7040) 
243it [00:11, 12.79it/s]d pid=104) 
244it [00:11, 11.82it/s]d pid=12564) 
243it [00:11, 12.27it/s]d 

322it [00:15, 14.53it/s]d pid=12500) 
322it [00:15, 13.06it/s]d pid=4112) 
324it [00:15, 12.70it/s]d pid=11412) 
319it [00:15, 14.02it/s]d pid=7192) 
322it [00:15, 13.74it/s]d pid=14252) 
319it [00:15, 13.91it/s]d pid=12220) 
321it [00:15, 14.46it/s]d pid=13960) 
326it [00:15, 13.48it/s]d pid=11412) 
322it [00:15, 13.83it/s]d pid=104) 
323it [00:16, 12.46it/s]d pid=12564) 
322it [00:15, 13.73it/s]d pid=7040) 
324it [00:15, 12.97it/s]d pid=13100) 
324it [00:15, 13.45it/s]d pid=12500) 
321it [00:16, 14.26it/s]d pid=12220) 
324it [00:16, 11.65it/s]d pid=4112) 
325it [00:16, 13.49it/s]d pid=12564) 
326it [00:16, 14.20it/s]d pid=12500) 
322it [00:16, 14.07it/s]d pid=7192) 
324it [00:16, 12.33it/s]d pid=14252) 
323it [00:16, 12.98it/s]d pid=13960) 
328it [00:16, 11.85it/s]d pid=11412) 
326it [00:16, 13.34it/s]d pid=13100) 
326it [00:16, 12.46it/s]d pid=4112) 
324it [00:16, 12.99it/s]d pid=104) 
324it [00:16, 12.58it/s]d pid=7040) 
326it [00:16, 12.97it/s]d pid=14252) 
323it [00:16, 12.85it/s

417it [00:21, 11.25it/s]d pid=7040) 
419it [00:21, 11.35it/s]d pid=13100) 
419it [00:21, 10.51it/s]d pid=12500) 
417it [00:21, 10.96it/s]d pid=4112) 
417it [00:21, 11.42it/s]d pid=13960) 
424it [00:21, 12.98it/s]d pid=11412) 
417it [00:21, 11.27it/s]d pid=104) 
422it [00:21, 12.38it/s]d pid=12500) 
419it [00:21, 10.78it/s]d pid=12564) 
422it [00:21, 13.32it/s]d pid=13100) 
424it [00:21, 13.01it/s]d pid=12500) 
418it [00:21, 11.03it/s]d pid=14252) 
403it [00:21, 10.49it/s]d pid=7152) 
419it [00:21, 10.99it/s]d pid=13960) 
419it [00:21, 11.01it/s]d pid=104) 
422it [00:21, 12.89it/s]d pid=12564) 
420it [00:21, 11.60it/s]d pid=7040) 
427it [00:22, 12.82it/s]d pid=11412) 
424it [00:21, 13.29it/s]d pid=13100) 
417it [00:21, 10.77it/s]d pid=7192) 
420it [00:22, 11.64it/s]d pid=14252) 
417it [00:21, 10.83it/s]d pid=12220) 
405it [00:21, 10.23it/s]d pid=7152) 
419it [00:21, 10.00it/s]d pid=4112) 
422it [00:22, 12.88it/s]d pid=13960) 
422it [00:21, 13.01it/s]d pid=104) 
424it [00:22, 12.95it/s]d

474it [00:25, 10.87it/s]d pid=12500) 
464it [00:25,  8.74it/s]d pid=7192) 
471it [00:25, 11.91it/s]d pid=14252) 
468it [00:26, 11.18it/s]d pid=12220) 
458it [00:25, 12.00it/s]d pid=7152) 
471it [00:25, 12.25it/s]d pid=4112) 
472it [00:26, 10.78it/s]d pid=13960) 
474it [00:25, 11.39it/s]d pid=11412) 
471it [00:25, 11.52it/s]d pid=104) 
474it [00:26, 10.77it/s]d pid=13100) 
473it [00:26,  9.76it/s]d pid=12564) 
470it [00:25,  9.42it/s]d pid=7040) 
465it [00:26,  7.16it/s]d pid=7192) 
473it [00:26, 10.70it/s]d pid=14252) 
473it [00:26, 10.34it/s]d pid=4112) 
485it [00:26, 22.23it/s]d pid=11412) 
485it [00:26, 21.08it/s]d pid=12500) 
473it [00:26, 10.34it/s]d pid=104) 
475it [00:26, 10.86it/s]d pid=12564) 
472it [00:26, 10.12it/s]d pid=7040) 
485it [00:26, 19.87it/s]d pid=13100) 
468it [00:26,  9.46it/s]d pid=7192) 
475it [00:26, 11.12it/s]d pid=14252) 
470it [00:26,  9.46it/s]d pid=12220) 
475it [00:26, 10.50it/s]d pid=4112) 
474it [00:26,  9.04it/s]d pid=13960) 
460it [00:26,  9.46it/s]d

...

71700it [12:31, 49.85it/s]pid=13100) 
69309it [12:30, 217.67it/s]id=7152) 
71708it [12:31, 46.60it/s]pid=4112) 
71590it [12:31, 305.80it/s]id=13960) 
71821it [12:31, 41.54it/s]pid=11412) 
71664it [12:31, 114.04it/s]id=7192) 
71747it [12:31, 29.47it/s]pid=104) 
71729it [12:31, 37.29it/s]pid=12500) 
71738it [12:31, 26.93it/s]pid=14252) 
71714it [12:31, 44.76it/s]pid=4112) 
71708it [12:31, 48.09it/s]pid=13100) 
71700it [12:31, 49.85it/s]pid=12564) 
71678it [12:31, 101.75it/s]id=7192) 
69349it [12:31, 194.94it/s]id=7152) 
71721it [12:31, 44.00it/s]pid=4112) 
71620it [12:31, 130.96it/s]id=7040) 
71714it [12:31, 46.27it/s]pid=13100) 
71733it [12:31, 30.98it/s]pid=12500) 
71747it [12:31, 28.62it/s]pid=14252) 
71708it [12:31, 48.99it/s]pid=12564) 
71825it [12:31, 28.93it/s]pid=11412) 
71721it [12:31, 44.53it/s]pid=13100) 
69381it [12:31, 178.30it/s]id=7152) 
71642it [12:31, 128.46it/s]id=7040) 
71751it [12:31, 18.04it/s]pid=104) 
71714it [12:31, 47.41it/s]pid=12564) 
71738it [12:31, 27.56it/s]

72353it [12:37, 203.86it/s]id=4112) 
71815it [12:37, 58.69it/s]pid=13960) 
70304it [12:37, 133.69it/s]id=7152) 
72703it [12:37, 115.72it/s]id=11412) 
72318it [12:37, 285.91it/s]id=12564) 
71825it [12:37, 28.04it/s]pid=7040) 
71944it [12:37, 82.68it/s]pid=7192) 
72374it [12:37, 98.43it/s] id=14252) 
70940it [12:37, 68.43it/s]pid=12220) 
70442it [12:37, 207.12it/s]id=7152) 
72350it [12:37, 198.62it/s]id=13100) 
72720it [12:37, 108.25it/s]id=11412) 
72393it [12:37, 81.66it/s]pid=104) 
71956it [12:37, 78.48it/s]pid=7192) 
70948it [12:37, 58.65it/s]pid=12220) 
71821it [12:37, 42.03it/s]pid=13960) 
72903it [12:37, 290.67it/s]id=11412) 
70493it [12:37, 209.24it/s]id=7152) 
72428it [12:37, 91.79it/s]pid=104) 
72349it [12:38, 193.19it/s]id=12564) 
71913it [12:37, 78.69it/s]pid=7040) 
72042it [12:37, 139.03it/s]id=7192) 
72392it [12:38, 79.14it/s]pid=14252) 
70996it [12:38, 85.47it/s]pid=12220) 
72375it [12:38, 102.59it/s]id=12500) 
71944it [12:38, 86.25it/s]pid=7040) 
72589it [12:38, 190.96it/s

73512it [12:44, 150.19it/s]id=14252) 
70907it [12:44, 56.35it/s]pid=7152) 
73489it [12:44, 144.77it/s]id=12500) 
72703it [12:44, 116.42it/s]id=13960) 
73802it [12:44, 232.45it/s]id=104) 
73425it [12:44, 249.38it/s]id=12564) 
74074it [12:44, 181.51it/s]id=11412) 
72998it [12:44, 122.11it/s]id=7192) 
71690it [12:44, 66.39it/s]pid=12220) 
70940it [12:44, 70.46it/s]pid=7152) 
72884it [12:44, 274.52it/s]id=13960) 
73703it [12:44, 211.81it/s]id=14252) 
73473it [12:44, 121.73it/s]id=4112) 
74146it [12:44, 198.54it/s]id=11412) 
73703it [12:44, 224.61it/s]id=12500) 
73040it [12:45, 127.52it/s]id=7192) 
71699it [12:45, 60.69it/s]pid=12220) 
70949it [12:44, 58.91it/s]pid=7152) 
72951it [12:45, 269.96it/s]id=13960) 
73004it [12:45, 129.15it/s]id=7040) 
73072it [12:45, 128.27it/s]id=7192) 
73802it [12:45, 222.73it/s]id=14252) 
71706it [12:45, 56.33it/s]pid=12220) 
73802it [12:45, 235.87it/s]id=12500) 
73842it [12:45, 163.18it/s]id=104) 
73470it [12:45, 125.41it/s]id=13100) 
73136it [12:45, 161.51it

74200it [12:52, 69.41it/s]pid=7192) 
71706it [12:52, 50.64it/s]pid=7152) 
74628it [12:52, 54.97it/s]pid=11412) 
74486it [12:52, 160.91it/s]id=12564) 
74643it [12:53, 55.05it/s]pid=11412) 
74313it [12:53, 126.34it/s]id=7192) 
74167it [12:52, 64.82it/s] id=7040) 
71712it [12:53, 38.11it/s]pid=7152) 
74506it [12:53, 47.53it/s] id=104) 
72372it [12:53, 90.33it/s] id=12220) 
74182it [12:53, 65.67it/s]pid=7040) 
71721it [12:53, 39.32it/s]pid=7152) 
74702it [12:53, 73.87it/s]pid=11412) 
74347it [12:53, 120.79it/s]id=7192) 
74608it [12:53, 79.35it/s]pid=104) 
74200it [12:53, 68.47it/s]pid=7040) 
71729it [12:53, 35.80it/s]pid=7152) 
74718it [12:53, 72.60it/s]pid=11412) 
74506it [12:53, 49.43it/s] id=14252) 
72389it [12:53, 73.90it/s]pid=12220) 
74385it [12:53, 113.12it/s]id=7192) 
74313it [12:53, 127.18it/s]id=7040) 
71733it [12:53, 30.29it/s]pid=7152) 
74506it [12:53, 47.15it/s] id=12500) 
74608it [12:54, 82.30it/s]pid=14252) 
72428it [12:54, 88.29it/s]pid=12220) 
74730it [12:54, 60.10it/s]pid

75660it [13:01, 69.28it/s] id=11412) 
75555it [13:01, 187.16it/s]id=12500) 
75618it [13:01, 174.72it/s]id=14252) 
75469it [13:01, 332.90it/s]id=12564) 
74969it [13:01, 84.38it/s]pid=7192) 
72390it [13:01, 71.94it/s]pid=7152) 
74730it [13:01, 56.82it/s]pid=13960) 
72428it [13:01, 84.79it/s]pid=7152) 
74984it [13:02, 76.67it/s]pid=7192) 
75546it [13:02, 186.96it/s]id=4112) 
74829it [13:02, 113.05it/s]id=13960) 
75697it [13:02, 67.37it/s]pid=11412) 
75625it [13:02, 106.34it/s]id=104) 
74906it [13:02, 70.79it/s] id=7040) 
75200it [13:02, 284.49it/s]id=7192) 
72589it [13:02, 181.48it/s]id=7152) 
73865it [13:02, 137.00it/s]id=12220) 
74887it [13:02, 136.82it/s]id=13960) 
75708it [13:02, 96.76it/s]pid=11412) 
74950it [13:02, 89.57it/s]pid=7040) 
75529it [13:02, 185.18it/s]id=13100) 
72652it [13:02, 199.92it/s]id=7152) 
75603it [13:02, 170.90it/s]id=4112) 
75513it [13:02, 182.22it/s]id=12564) 
73916it [13:02, 145.03it/s]id=12220) 
74969it [13:02, 85.78it/s]pid=7040) 
75272it [13:02, 218.69it/s

Ray 사용에 대해서는 분산처리 설계가 좀 더 세부적으로 이뤄져야 할 것 같다

전역 변수에 대해서 공통적으로 자료를 입력하는 식으로는 Ray 라이브러리를 사용할 수 없었다

In [ ]:
## std_dict를 피클화해서 저장. 이후에 변환 속도를 빠르게 해줌
with open('std_dict.pkl','wb') as f:
  pickle.dump(std_dict,f)

## OOV 빈도수 사전을 저장
oov_df = pd.DataFrame({"OOV_count": OOV_list})
oov_df['OOV_menu'] = oov_df.index
oov_df.reset_index(drop=True, inplace=True)
oov_df.to_csv("OOV_count_list.csv", index=False)

## OOV의 표준화 결과를 저장
std_df = pd.DataFrame(std_oov_list, columns=['token', 'token_std'])
std_df.to_csv("OOV_std_list.csv", index=False)

## 분류된 메뉴를 DataFrame화 
df_menu = pd.DataFrame(columns=['title', 'std_title'], data=menu_cluster.items())
df_menu.to_csv("menu_std.csv")

In [ ]:
len(menu_cluster)

75708

### 초/중/종성까지 전부 고려한 표준화

In [ ]:
menu_cluster

TO-DO

- [해결] 표준화 적용 대상 줄이기

-> 전체 적용시 비슷한 단어가 섞임

- [해결] OOV_list 중복값 제거, 딕셔너리화 고려

- [해결] 일련의 파이프라인에서 향후 EDA를 적용할 미분류 값들에 대한 정의

In [ ]:
len(OOV_list)

24908

In [ ]:
# 분류된 메뉴를 DataFrame화 
df_menu = pd.DataFrame(columns=['title', 'category'], data=menu_cluster.items())
df_menu['category'].unique()

array(['-', '파스타 구구 닥다리', '디카페인', ..., '타로 앙고기 리조토', '사슴힘줄', '쇠고기저키'],
      dtype=object)

In [ ]:
len(df_menu['category'].unique())

36068

In [ ]:
df_copy

In [ ]:
df_menu['표준상품명'] = df_copy['표준상품명']

In [ ]:
for i, category in enumerate(df_menu['표준상품명']):
  if category == '-':
    df_menu['category'][i] = df_menu['title'][i]

In [ ]:
df_menu.to_csv("train_df_v01.csv", index=False)

In [ ]:
pd.set_option('display.max_row', 100)

In [ ]:
df_menu.sample(100)

,title,category
69774,팔도틈새매운카레,팔도 틈세 메운 카레
42794,소시지 함박 스테이크,소세지 함박 스테이크
8277,50g추가,-
41512,생크림 누텔라 오레오 와플,셍크림 누텔라 오레오 아플
48710,핫 얼그레이 밀크티,핫 얼그레이 밀크티
73495,[더리터] ICE 헤이즐넛 라떼 Liter,헤이즐넛 라테
63300,I.카페모카(이벤트),카페모카
26641,라면 국물,라먼 국물
57627,청포도플레시에이드,청포도 에이드
37339,부라타치즈샐러드,부라타 치즈 셀러드


### 생각할 부분

- 예외처리 부분 + OOV로 분류하기 전에 표준화